# Daten einlesen und Pipeline definieren

In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.metrics import r2_score

# 1. Daten einlesen und zusammenführen
df1 = pd.read_csv('../umsatzdaten_gekuerzt.csv')
df2 = pd.read_csv('../wetter.csv')
df3 = pd.read_csv('../kiwo.csv')
df4 = pd.read_csv('../Feier_Bruecke_Ferien_bis2018.csv')
df5 = pd.read_csv('../VPI.csv')

df = df1.merge(df2, on='Datum', how='left')
df = df.merge(df3, on='Datum', how='left')
df = df.merge(df4, on='Datum', how='left')
df = df.merge(df5, on='Datum', how='left')

# Sicherstellen, dass 'Datum' als datetime konvertiert ist
df['Datum'] = pd.to_datetime(df['Datum'])
df.sort_values(by='Datum')


# 2. Pipeline-Komponenten definieren

# a) Feature Engineering: Erstellen von Datum- und zyklischen Features
def add_features(df):
    df = df.copy()
    # Basismerkmale
    df['Jahr'] = df['Datum'].dt.year
    df['Monat'] = df['Datum'].dt.month
    df['Wochentag'] = df['Datum'].dt.weekday
    df['Kalenderwoche'] = df['Datum'].dt.isocalendar().week
    df['Tag_im_Jahr'] = df['Datum'].dt.dayofyear
    df['Ist_Wochenende'] = df['Wochentag'].isin([5, 6]).astype(int)
    
    # Zyklische Merkmale
    df['Tag_im_Jahr_sin'] = np.sin(2 * np.pi * df['Tag_im_Jahr'] / 365)
    df['Tag_im_Jahr_cos'] = np.cos(2 * np.pi * df['Tag_im_Jahr'] / 365)
    df['Monat_sin'] = np.sin(2 * np.pi * df['Monat'] / 12)
    df['Monat_cos'] = np.cos(2 * np.pi * df['Monat'] / 12)
    df['Wochentag_sin'] = np.sin(2 * np.pi * df['Wochentag'] / 7)
    df['Wochentag_cos'] = np.cos(2 * np.pi * df['Wochentag'] / 7)
    
    # Entfernen der 'Datum'-Spalte nach der Feature-Erstellung
    df = df.drop('Datum', axis=1)
    
    return df

# b) FunctionTransformer für Feature Engineering
feature_engineering = FunctionTransformer(add_features, validate=False)

# c) Definieren der Feature-Gruppen
numeric_features = ['Temperatur', 'Bewoelkung', 'Windgeschwindigkeit',
                    'Tag_im_Jahr_sin', 'Tag_im_Jahr_cos',
                    'Monat_sin', 'Monat_cos',
                    'Wochentag_sin', 'Wochentag_cos', 'KielerWoche'
                    'feiertag', 'brueckentag', 'BW', 'BY', 'B', 
                    'BB', 'HB', 'HH', 'HE', 'MV', 'NI', 'NW', 'RP', 
                    'SL', 'SN', 'ST', 'SH', 'TH', 'VPI_brot_getreide']
                    
categorical_features = ['Warengruppe', 'Wettercode', 'Wochentag']

# d) Definieren des ColumnTransformers
# Stellen Sie sicher, dass 'Wettercode' als String vorliegt
df['Wettercode'] = df['Wettercode'].astype(str)

# Liste der Kategorien für 'Wettercode'
wettercode_categories = [str(i) for i in range(0, 100)] + ['Unbekannt']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler())
        ]), numeric_features),
        
        ('wettercode', OneHotEncoder(categories=[wettercode_categories], handle_unknown='ignore'), ['Wettercode']),
        
        ('warengruppe', OneHotEncoder(handle_unknown='ignore'), ['Warengruppe']),
        
        ('wochentag', OneHotEncoder(handle_unknown='ignore'), ['Wochentag'])
    ],
    remainder='drop'
)



# Define your date thresholds
train_end_date = '2017-07-31'
validation_end_date = '2018-08-31'
# Convert to datetime if not already

# Split the data based on the date thresholds

train_data = df[df['Datum'] <= train_end_date]
test_data = df[df['Datum'] > train_end_date]


X_train  = train_data.drop(['Umsatz'], axis=1)  # Behalten Sie 'Datum' in X für die Pipeline
y_train = train_data['Umsatz']

X_test  = test_data.drop(['Umsatz'], axis=1)  # Behalten Sie 'Datum' in X für die Pipeline
y_test = test_data['Umsatz']



#erste fünf Zeilen anzeigen
print(df.head())



        id      Datum  Warengruppe      Umsatz  Bewoelkung  Temperatur  \
0  1307011 2013-07-01            1  148.828353         6.0     17.8375   
1  1307021 2013-07-02            1  159.793757         3.0     17.3125   
2  1307031 2013-07-03            1  111.885594         7.0     21.0750   
3  1307041 2013-07-04            1  168.864941         7.0     18.8500   
4  1307051 2013-07-05            1  171.280754         5.0     19.9750   

   Windgeschwindigkeit Wettercode  KielerWoche  feiertag  ...  SL  SN  ST  SH  \
0                 15.0       20.0          NaN         0  ...   0   0   0   1   
1                 10.0        nan          NaN         0  ...   0   0   0   1   
2                  6.0       61.0          NaN         0  ...   0   0   0   1   
3                  7.0       20.0          NaN         0  ...   0   0   0   1   
4                 12.0        nan          NaN         0  ...   0   0   0   1   

   TH  VPI_brot_getreide  VPI_molkerei_ei  VPI_speiseoel_fett  VPI_O